# Catalytic disruption analysis
In this script, we assess the extent to which functional annotation of catalytic edges in the *i*CH360 knowledge graph can enhance phenotypic predictions. To this end, we focus on exploring the *in silico* knockout of all genes associated to model essential reactions (i.e. reactions required to produce biomass in the model). We use the knowledge graph to classify the qualitative outcome of each knockout, in terms of the catalytic disruption it causes. Finally, we compare this *in silico* predicted classes with experimental measurements of the mutant fitness (relative to WT) using the databse of mutant fitness data from [1]:

1. Price, M. N. et al. Mutant phenotypes for thousands of bacterial genes of unknown function. Nature 557, 503–509 (2018).

## Imports

In [7]:
import networkx as nx
import pandas as pd
from functools import reduce
from pyvis.network import Network
import sys
import cobra
sys.path.append('../../utils')
import graph_utils
import importlib
import re
importlib.reload(graph_utils)

<module 'graph_utils' from 'c:\\Users\\marco\\repos\\ich360_manuscript\\Analysis\\catalytic_disruption_analysis\\../../utils\\graph_utils.py'>

## Load stoichiometric model and knowledge graph

In [8]:
model=cobra.io.read_sbml_model('../../Model/iCH360/Escherichia_coli_iCH360.xml')
graph=nx.read_gml('../../Knowledge_graph/ich360_graph.gml')

'' is not a valid SBML 'SId'.


## Some useful functions

In [9]:
def is_essential(model,r_id,carbon_source='glc__D',condition='aerobic',tr=0.05):
    """
    Determines whether a reaction is essential for a given growth condition (carbon source + oxygen availability)
    """
    with model as m:
        #Set carbon source
        m.reactions.get_by_id(f'EX_{carbon_source}_e').lower_bound=-10
        #Set condition
        if condition=='aerobic':
            m.reactions.get_by_id('EX_o2_e').lower_bound=-1000
        elif condition=='anaerobic':
            m.reactions.get_by_id('EX_o2_e').lower_bound=0
        else:
            raise ValueError('condition must be either aerobic or anaerobic')
        if r_id not in model.reactions:
            print(f'{r_id} not in model')
            return False
        m.reactions.get_by_id(r_id).knock_out()
        sol=model.optimize()
        if sol.status=='infeasible':
            return True
        elif sol.objective_value<tr:
            return True
        else:
            return False
        
def is_spontaneous(reaction):
    """
    flags spontaneous (non-enzymatic) reactions
    """
    if 's0001' in model.reactions.get_by_id(reaction).gene_reaction_rule:
        return True
    else:
        return False

## Classifying disruption outcomes arising from single gene KOs in the network

In [10]:
%%capture
#enumerate all enzymes in the graph
enzyme_nodes=graph_utils.compute_catalytic_nodes(graph)
#enumerate all genes
gene_nodes=[node for node in graph.nodes if graph.nodes[node]['type']=='gene']
#For each enzyme, pool all genes that are associated to it
enzyme_gene_map={node:graph_utils.genes_in_gpr(graph_utils.compute_node_gpr(graph,node)) for node in enzyme_nodes}
carbon_sources=['glc__D','fru','lac__D','pyr','succ','ac','glyc','rib__D','xyl__D']
essential_reactions={carbon_source:[r.id for r in model.reactions if is_essential(model,r.id,carbon_source=carbon_source)] 
                      for carbon_source in carbon_sources}


Loop across coditions. For each conditions, loop across genes in the graph and, if a gene is associated with an essential reaction, assess the effect of its knockout in terms of catalytic dsruption.

In [15]:
%%capture
import tqdm
reaction_nodes=[node for node in graph.nodes if graph.nodes[node]['type']=='reaction']

all_dfs=[]
for carbon_source in tqdm.tqdm(carbon_sources):
    ko_df=pd.DataFrame(columns=['carbon_source',
                            'primary_catalysis_loss',
                            'secondary_catalysis_loss',
                            'complete_primary',
                            'complete'
                            ])
    for gene in (gene_nodes):
        enzymes_for_this_gene=list(set([enzyme for enzyme in enzyme_gene_map.keys() if gene in enzyme_gene_map[enzyme]]))
        if len(enzymes_for_this_gene)==0:
            continue
        temp_graph=graph.copy()
        #Which reactions are disrupted by KO of this gene?
        disrupted_reactions=reduce(lambda x,y: x+y,[list(temp_graph.predecessors(enzyme)) for enzyme in enzymes_for_this_gene])
        #Remove from the graph the enzymes for which the gene is required 
        temp_graph.remove_nodes_from(enzymes_for_this_gene)

        #Initialise counters
        primary_catalysis_loss=0
        secondary_catalysis_loss=0
        ko_df.loc[gene,'carbon_source']=carbon_source
        ko_df.loc[gene,'complete_primary']=False
        ko_df.loc[gene,'complete']=False


        for reaction in disrupted_reactions:
            #If the reaction is not essential for this condition, or is spontaneous, skip
            if reaction.replace('bigg:','') not in essential_reactions[carbon_source] \
                or \
            is_spontaneous(reaction.replace('bigg:','')):
               continue
            #Else, assess the disruption causes
            #Edges types for this reaction prior to KO
            init_edge_types=[graph.edges[edge]['subtype'] for edge in graph.edges(reaction)]
            #Edges types for this reaction after KO
            final_edges_types=[temp_graph.edges[edge]['subtype'] for edge in temp_graph.edges(reaction)]
            #Counting losses
            lost_primary_catalysis=init_edge_types.count('primary')-final_edges_types.count('primary')
            primary_catalysis_loss+=lost_primary_catalysis
            lost_secondary_catalysis=init_edge_types.count('secondary')-final_edges_types.count('secondary')
            secondary_catalysis_loss+=lost_secondary_catalysis


            remaining_primary=final_edges_types.count('primary')
            remaining_overall=len(final_edges_types)
            #Did any reaction lost all its primary catalytic edges?
            if remaining_primary==0:
                ko_df.loc[gene,'complete_primary']=True

            #Did any reaction lost all of its catalytic edges
            if remaining_overall==0:
                ko_df.loc[gene,'complete']=True

        ko_df.loc[gene,'primary_catalysis_loss']=primary_catalysis_loss
        ko_df.loc[gene,'secondary_catalysis_loss']=secondary_catalysis_loss
    all_dfs.append(ko_df.reset_index().rename(columns={'index':'gene'}))
    
ko_df=pd.concat(all_dfs,axis=0)

ko_df.head()

Now classify overall disruption outcome as one of:
- complete disruption (all catalytic edges lost by a reaction)
- primary full disruption (all primary catalys edges lost by a reaction, but secondary remaining)
- primary partial disruption (some, but not all primary edges have been disrupted)
- secondary disruption (only secondary edges affected)
If multiple reactions are affected by a KO in different way, we take the overall disruption to be the one expected to be "most disruptive" among them, using the above order of precedence (from most to least disruptive)

In [23]:
#Overall disruptions======================
overall_disruption_df=ko_df.copy().drop(columns=['complete','complete_primary'])

#Overall disruption is complete if KO caused at least one complete disruption
overall_disruption_df.loc[ko_df['complete'],'overall_disruption']='complete'
#Overall disruption is complete primary if KO caused complete primary, but not complete disruption
overall_disruption_df.loc[(ko_df['complete_primary'])&(~ko_df['complete']),'overall_disruption']='primary_full'
#Overall disruption is partial primary if KO caused loss of some primary edges, but no reaction was flagged as having lost all its primary edges
overall_disruption_df.loc[(~ko_df['complete_primary'])&(ko_df['primary_catalysis_loss']>0),'overall_disruption']='primary_partial'
#Overall disruption is secondary if it casued no primary loss, but some secondary loss
overall_disruption_df.loc[(ko_df['primary_catalysis_loss']==0)&
                          (ko_df['secondary_catalysis_loss']>0),'overall_disruption']='secondary'
overall_disruption_df.set_index(['gene','carbon_source'],inplace=True)

In [24]:
overall_disruption_df.head()

,,primary_catalysis_loss,secondary_catalysis_loss,overall_disruption
gene,carbon_source,,,
b0474,glc__D,1,7,complete
b2518,glc__D,7,0,primary_full
b3281,glc__D,1,0,primary_full
b1062,glc__D,1,0,complete
b1281,glc__D,1,0,complete


## Validation against experimental mutatant fitness data

In [26]:
#Read fitness data from price et.al 2018
fitness_data=pd.read_csv('./mutant_fitness_data/fit_organism_Keio.tsv',sep='\t',index_col='sysName')
#read metadata mapping each experiments to carbon sources in the model
experiment_carbon_source_map=pd.read_csv('./mutant_fitness_data/experiment_cabon_source_map.csv')
conditions=experiment_carbon_source_map['experiment']
experiment_carbon_source_map.head()

,experiment,carbon_source
0,set1IT003,glc__D
1,set1IT004,glc__D
2,set2IT094,glc__D
3,set2IT096,glc__D
4,set1IT005,fru


Parse the experimental data and mapped it to the classification of catalytic disruption we performed before

In [27]:
fitness_data.columns=[re.sub(' [a-z()A-Z-\d]*','',c) for c in fitness_data.columns]
fitness_data=fitness_data.loc[:,conditions]
fitness_data=fitness_data.dropna()
fitness_data.reset_index(inplace=True,names='gene')
fitness_data=fitness_data.melt(id_vars='gene',var_name='experiment',value_name='fitness')
fitness_data['carbon_source']=fitness_data['experiment'].map(experiment_carbon_source_map.set_index('experiment')['carbon_source'])

for i,row in fitness_data.iterrows():
    gene=row['gene']
    carbon_source=row['carbon_source']
    if (gene,carbon_source) in overall_disruption_df.index:
        fitness_data.loc[i,'overall_disruption']=overall_disruption_df.loc[(gene,carbon_source),'overall_disruption'] 
    else:
        fitness_data.loc[i,'overall_disruption']=pd.NA
fitness_data=fitness_data.dropna()
fitness_data.set_index(['gene','experiment'],inplace=True)
fitness_data.head(2)

,,fitness,carbon_source,overall_disruption
gene,experiment,,,
b0002,set1IT003,-3.459,glc__D,primary_full
b0003,set1IT003,-2.971,glc__D,complete


In [11]:
#average across replicate for the same gene-condition pair
validation_df=fitness_data.groupby(['gene','carbon_source']).mean()
validation_df['overall_disruption']=overall_disruption_df.loc[validation_df.index,'overall_disruption']

C:\Users\marco\AppData\Local\Temp\ipykernel_82952\2625274586.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  validation_df=fitness_data.groupby(['gene','carbon_source']).mean()


We exclude the following genes from the analysis:
- b2903,b2904, b2905: components of the Glycine cleavege system. This is a known false essential reaction in iCH360.
- b4005. This is required for thioredoxin regeneration. In iCH360 a number of alternative reactions using glutaredozxin instead of thioredoxin as an electron donor are not included, making thioredoxin regeneration essential in the model, but not in practice


In [12]:
genes_to_remove=['b2903','b2904','b2905','b0888']
validation_df=validation_df.query("gene not in @genes_to_remove")

## Export data for plotting (see `../Manuscript_Figures/notebooks/annotation_and_graph.Rmd`)


In [13]:
validation_df.to_csv("out/catalytic_disruption_analysis_post_processed_results.csv")